In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH_TEST_COLLECTION_DATA="drive/MyDrive/Uni/Master/Masterthesis/Data/test_collection/"

In [ ]:
!pip install -U sentence-transformers
!pip install -U qdrant-client
!pip install fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.2.0
    Uninstalling sentence-transformers-5.2.0:
      Successfully uninstalled sentence-transformers-5.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from sentence_transformers.sparse_encoder import SparseEncoder

# Load pooling data

In [ ]:
import json

# importieren der Pooling results
pooling_file = PATH_TEST_COLLECTION_DATA + "pooling_results_final.json"

# JSON direkt einlesen
with open(pooling_file, "r", encoding="utf-8") as f:
    pooling_results = json.load(f)

pooling_results[0]

{'doc_id': '67251b202f496742be0ea207',
 'doc_raw': '{"branches": [], "companyLocationCity": "Wuppertal", "companyLocationStreet": "Schwelmer Straße 245", "companyLocationZip": 42389.0, "companyName": "Salto Systems GmbH", "companyTypes": [], "companyWebsite": "saltosystems.com/de", "description": "Ich bin ein Enthusiast für elektronische Zutrittskontrolle und Schließsysteme und es begeistert mich, wenn durch integrative Lösungen Probleme gelöst und Prozesse optimiert werden können.", "employeeOfInstitutionNames": [], "firstName": "Bela", "gender": "MALE", "id": "67251b202f496742be0ea207", "jobTitle": "System- und Projektberater", "lastName": "Marahrens", "projectsDescription": "Elektronische Zutrittskontrolle\\nElektronische Schließsysteme\\nBesuchermanagement\\nCloudbasierte Zutrittskontrolle", "skills": [], "title": null, "full_text": "Bela Marahrens ist System- und Projektberater. Er ist Enthusiast für elektronische Zutrittskontrolle und Schließsysteme und überzeugt davon, dass inte

# Create Qdrant Collections

In [ ]:
client = QdrantClient(
    url="qdrant_url",
    api_key="qdrant_api_key",
)

## Create Collections BI-Encoder

In [ ]:
bi_encoder = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
if not client.collection_exists("bi_encoder_structured"):
  client.create_collection(
      collection_name="bi_encoder_structured",
      vectors_config=models.VectorParams(
          size=bi_encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
          distance=models.Distance.COSINE,
      ),
  )

if not client.collection_exists("bi_encoder_fulltext"):
  client.create_collection(
      collection_name="bi_encoder_fulltext",
      vectors_config=models.VectorParams(
          size=bi_encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
          distance=models.Distance.COSINE,
      ),
  )

if not client.collection_exists("bi_encoder_structured_fulltext"):
  client.create_collection(
      collection_name="bi_encoder_structured_fulltext",
      vectors_config=models.VectorParams(
          size=bi_encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
          distance=models.Distance.COSINE,
      ),
  )

## Create Collections Neural-Sparse

In [ ]:
if not client.collection_exists("neural_sparse_structured"):
  client.create_collection(
      collection_name="neural_sparse_structured",
      vectors_config={},
      sparse_vectors_config={
          "neural_sparse_vector": models.SparseVectorParams(),
      },
  )

if not client.collection_exists("neural_sparse_fulltext"):
  client.create_collection(
      collection_name="neural_sparse_fulltext",
      vectors_config={},
      sparse_vectors_config={
          "neural_sparse_vector": models.SparseVectorParams(),
      },
  )

if not client.collection_exists("neural_sparse_structured_fulltext"):
  client.create_collection(
      collection_name="neural_sparse_structured_fulltext",
      vectors_config={},
      sparse_vectors_config={
          "neural_sparse_vector": models.SparseVectorParams(),
      },
  )

## Create Collections Late-Interaction

In [ ]:
from fastembed import LateInteractionTextEmbedding

late_model = LateInteractionTextEmbedding("jinaai/jina-colbert-v2")
print(late_model.embedding_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/3.98M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

onnx/model.onnx_data:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

128


In [ ]:
from qdrant_client.models import VectorParams, Distance

if not client.collection_exists("late_interaction_structured"):
  client.create_collection(
      collection_name='late_interaction_structured',
      vectors_config={
          "late_interaction": models.VectorParams(
              size=late_model.embedding_size,
              distance=models.Distance.COSINE,
              multivector_config=models.MultiVectorConfig(
                  comparator=models.MultiVectorComparator.MAX_SIM
              ),
              hnsw_config=models.HnswConfigDiff(m=0),
          )
      },
  )

if not client.collection_exists("late_interaction_fulltext"):
  client.create_collection(
      collection_name='late_interaction_fulltext',
      vectors_config={
          "late_interaction": models.VectorParams(
              size=late_model.embedding_size,
              distance=models.Distance.COSINE,
              multivector_config=models.MultiVectorConfig(
                  comparator=models.MultiVectorComparator.MAX_SIM
              ),
              hnsw_config=models.HnswConfigDiff(m=0),
          )
      },
  )

if not client.collection_exists("late_interaction_structured_fulltext"):
  client.create_collection(
      collection_name='late_interaction_structured_fulltext',
      vectors_config={
          "late_interaction": models.VectorParams(
              size=late_model.embedding_size,
              distance=models.Distance.COSINE,
              multivector_config=models.MultiVectorConfig(
                  comparator=models.MultiVectorComparator.MAX_SIM
              ),
              hnsw_config=models.HnswConfigDiff(m=0),
          )
      },
  )

# Index documents

## Prepare data

In [ ]:
pooling_results[0]["profile_dict"].keys()

dict_keys(['branches', 'companyLocationCity', 'companyLocationStreet', 'companyLocationZip', 'companyName', 'companyTypes', 'companyWebsite', 'description', 'employeeOfInstitutionNames', 'firstName', 'gender', 'id', 'jobTitle', 'lastName', 'projectsDescription', 'skills', 'title', 'full_text'])

In [ ]:
### create copy without fulltext
import copy
import json

pooling_results_copy = copy.deepcopy(pooling_results)

pooling_results_without_fulltext = []

for obj in pooling_results_copy:
    new_obj = obj.copy()

    if "profile_dict" in new_obj and isinstance(new_obj["profile_dict"], dict):
        # 1) full_text entfernen
        new_profile_dict = {
            k: v for k, v in new_obj["profile_dict"].items()
            if k != "full_text"
        }

        # 2) doc_raw überschreiben
        new_doc_raw = json.dumps(
            new_profile_dict,
            ensure_ascii=False,
            default=str
        )

        new_obj["profile_dict"] = new_profile_dict
        new_obj["doc_raw"] = new_doc_raw

    pooling_results_without_fulltext.append(new_obj)

# Check
print(pooling_results_without_fulltext[0]["profile_dict"].keys())
print(pooling_results_without_fulltext[1]["doc_raw"])
print(len(pooling_results_without_fulltext))


dict_keys(['branches', 'companyLocationCity', 'companyLocationStreet', 'companyLocationZip', 'companyName', 'companyTypes', 'companyWebsite', 'description', 'employeeOfInstitutionNames', 'firstName', 'gender', 'id', 'jobTitle', 'lastName', 'projectsDescription', 'skills', 'title'])
{"branches": ["Technologie"], "companyLocationCity": "Siegen", "companyLocationStreet": "Sonnenstraße 33-35", "companyLocationZip": 57078.0, "companyName": "K-iS Systemhaus GmbH", "companyTypes": ["Dienstleistung"], "companyWebsite": "https://www.k-is.com/", "description": "Als Account Manager mit dem Hintergrund der Heilerziehungspflege bringe ich eine einzigartige Kombination aus Empathie und technischem Know-how mit. \nMeine Kernkompetenz ist es, komplexe Herausforderungen zu erkennen und in einfache Lösungen zu übersetzen.\nFlexibilität und Anpassungsfähigkeit sind die Skills, die ich auch gerne bei Ihnen anwende.\nIch stehe Ihnen als vertrauensvoller Partner zur Seite und unterstütze Sie bei der digital

In [ ]:
## create copy with only fulltext
import copy

pooling_results_copy = copy.deepcopy(pooling_results)

pooling_results_fulltext_only = []

for obj in pooling_results_copy:
    new_obj = obj.copy()

    profile = new_obj.get("profile_dict")

    if isinstance(profile, dict):
        # full_text extrahieren
        full_text = profile.get("full_text")

        # doc_raw entfernen
        new_obj.pop("doc_raw", None)

        # neuen Key setzen
        new_obj["doc_raw_fulltext"] = full_text

    pooling_results_fulltext_only.append(new_obj)

# Check
print(pooling_results_fulltext_only[0].keys())
print(pooling_results_fulltext_only[0])


dict_keys(['doc_id', 'profile_dict', 'doc_raw_fulltext'])
{'doc_id': '67251b202f496742be0ea207', 'profile_dict': {'branches': [], 'companyLocationCity': 'Wuppertal', 'companyLocationStreet': 'Schwelmer Straße 245', 'companyLocationZip': 42389.0, 'companyName': 'Salto Systems GmbH', 'companyTypes': [], 'companyWebsite': 'saltosystems.com/de', 'description': 'Ich bin ein Enthusiast für elektronische Zutrittskontrolle und Schließsysteme und es begeistert mich, wenn durch integrative Lösungen Probleme gelöst und Prozesse optimiert werden können.', 'employeeOfInstitutionNames': [], 'firstName': 'Bela', 'gender': 'MALE', 'id': '67251b202f496742be0ea207', 'jobTitle': 'System- und Projektberater', 'lastName': 'Marahrens', 'projectsDescription': 'Elektronische Zutrittskontrolle\nElektronische Schließsysteme\nBesuchermanagement\nCloudbasierte Zutrittskontrolle', 'skills': [], 'title': None, 'full_text': 'Bela Marahrens ist System- und Projektberater. Er ist Enthusiast für elektronische Zutrittsk

## Index documents bi-encoder

In [ ]:
## load only the structured original expert data without additional full_text
client.upload_points(
    collection_name="bi_encoder_structured",
    points=[
        models.PointStruct(
            id=idx,
            vector=bi_encoder.encode(doc["doc_raw"]).tolist(),
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
        for idx, doc in enumerate(pooling_results_without_fulltext)
    ],
)

In [ ]:
#load only the generated fulltext for each expert object
client.upload_points(
    collection_name="bi_encoder_fulltext",
    points=[
        models.PointStruct(
            id=idx,
            vector=bi_encoder.encode(doc["doc_raw_fulltext"]).tolist(),
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
        for idx, doc in enumerate(pooling_results_fulltext_only)
    ],
)

In [ ]:
#load the structured expert data with fulltext
client.upload_points(
    collection_name="bi_encoder_structured_fulltext",
    points=[
        models.PointStruct(
            id=idx,
            vector=bi_encoder.encode(doc["doc_raw"]).tolist(),
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
        for idx, doc in enumerate(pooling_results)
    ],
)

## Index documents neural-sparse

In [ ]:
# Download from the 🤗 Hub
sparse_model = SparseEncoder("opensearch-project/opensearch-neural-sparse-encoding-multilingual-v1")

modules.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

router_config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

./model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

In [ ]:
import torch

def encode_sparse_for_qdrant(text: str) -> models.SparseVector:

    tensor = sparse_model.encode_document(text)
    tensor = tensor.coalesce()

    idx = tensor.indices()   # Shape: [1, nnz]
    vals = tensor.values()   # Shape: [nnz]

    token_indices = idx[0].tolist()
    token_values = vals.tolist()

    return models.SparseVector(
        indices=token_indices,
        values=token_values,
    )

In [ ]:
## load only the structured original expert data without additional full_text
points_structured = []

for idx, doc in enumerate(pooling_results_without_fulltext):
    text = doc["doc_raw"]
    sparse_vec = encode_sparse_for_qdrant(text)

    points_structured.append(
        models.PointStruct(
            id=idx,
            vector={
                "neural_sparse_vector": sparse_vec,
            },
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
    )

client.upload_points(
    collection_name="neural_sparse_structured",
    points=points_structured,
)

In [ ]:
#load only the generated fulltext for each expert object
points_fulltext = []

for idx, doc in enumerate(pooling_results_fulltext_only):
    text = doc["doc_raw_fulltext"]
    sparse_vec = encode_sparse_for_qdrant(text)

    print(idx)

    points_fulltext.append(
        models.PointStruct(
            id=idx,
            vector={
                "neural_sparse_vector": sparse_vec,
            },
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
    )

client.upload_points(
    collection_name="neural_sparse_fulltext",
    points=points_fulltext,
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


In [ ]:


points_structured_fulltext = []

for idx, doc in enumerate(pooling_results):
    text = doc["doc_raw"]
    sparse_vec = encode_sparse_for_qdrant(text)

    print(idx)

    points_structured_fulltext.append(
        models.PointStruct(
            id=idx,
            vector={
                "neural_sparse_vector": sparse_vec,
            },
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
    )

client.upload_points(
    collection_name="neural_sparse_structured_fulltext",
    points=points_structured_fulltext,
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


## Index documents late-interaction

In [ ]:
## load only the structured original expert data without additional full_text
points_structured = []

for idx, doc in enumerate(pooling_results_without_fulltext):
    text = doc["doc_raw"]
    multivector = next(late_model.embed(text))

    print(idx)

    points_structured.append(
        models.PointStruct(
            id=idx,
            vector={
                "late_interaction": multivector,
            },
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
    )

client.upload_points(
    collection_name="late_interaction_structured",
    points=points_structured,
    batch_size=32,
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


In [ ]:
## load only the fulltext
points_fulltext = []

for idx, doc in enumerate(pooling_results_fulltext_only):
    text = doc["doc_raw_fulltext"]
    multivector = next(late_model.embed(text))

    print(idx)

    points_fulltext.append(
        models.PointStruct(
            id=idx,
            vector={
                "late_interaction": multivector,
            },
            payload={
                "doc_id": doc["doc_id"],
                "profile_dict": doc["profile_dict"],
            },
        )
    )

client.upload_points(
    collection_name="late_interaction_fulltext",
    points=points_fulltext,
    batch_size=32,
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


In [ ]:
pooling_results

batch_size = 10

batches = [
    pooling_results[i:i + batch_size]
    for i in range(0, len(pooling_results), batch_size)
]

len(batches)
for batch in batches:
  print(len(batch))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9


In [ ]:
#load the structured expert data with fulltext

idx = 0
for batch in batches: # split data into batches to avoid payload limit
  points_structured_fulltext = []
  for doc in batch:
      text = doc["doc_raw"]
      multivector = next(late_model.embed(text))

      print(idx)

      points_structured_fulltext.append(
          models.PointStruct(
              id=idx,
              vector={
                  "late_interaction": multivector,
              },
              payload={
                  "doc_id": doc["doc_id"],
                  "profile_dict": doc["profile_dict"],
              },
          )
      )

      idx += 1

  client.upload_points(
      collection_name="late_interaction_structured_fulltext",
      points=points_structured_fulltext,
      batch_size=32,
  )
  print("loaded data to db")

0
1
2
3
4
5
6
7
8
9
loaded data to db
10
11
12
13
14
15
16
17
18
19
loaded data to db
20
21
22
23
24
25
26
27
28
29
loaded data to db
30
31
32
33
34
35
36
37
38
39
loaded data to db
40
41
42
43
44
45
46
47
48
49
loaded data to db
50
51
52
53
54
55
56
57
58
59
loaded data to db
60
61
62
63
64
65
66
67
68
69
loaded data to db
70
71
72
73
74
75
76
77
78
79
loaded data to db
80
81
82
83
84
85
86
87
88
89
loaded data to db
90
91
92
93
94
95
96
97
98
99
loaded data to db
100
101
102
103
104
105
106
107
108
109
loaded data to db
110
111
112
113
114
115
116
117
118
119
loaded data to db
120
121
122
123
124
125
126
127
128
129
loaded data to db
130
131
132
133
134
135
136
137
138
139
loaded data to db
140
141
142
143
144
145
146
147
148
149
loaded data to db
150
151
152
153
154
155
156
157
158
159
loaded data to db
160
161
162
163
164
165
166
167
168
169
loaded data to db
170
171
172
173
174
175
176
177
178
179
loaded data to db
180
181
182
183
184
185
186
187
188
189
loaded data to db
190
191
